In [39]:
!pip install pandas numpy matplotlib statsmodels pandas_datareader datetime sklearn PyPortfolioOpt pandas-ta

  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [38]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import pandas_ta
import numpy as np
import datetime as dt
import yfinance as yf # Yahoo Finance
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500['Symbol'] = sp500['Symbol'].str.replace(".", "-")

symbols_list = sp500['Symbol'].unique().tolist()

end_date = "2024-08-03"
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)

ModuleNotFoundError: No module named 'pandas_ta'

In [11]:
start_date

Timestamp('2016-08-05 00:00:00')

In [28]:
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date
                 ).stack()

[*********************100%%**********************]  503 of 503 completed


In [15]:
df

Price        Adj Close                                     ...   Volume                           
Ticker               A        AAL        AAPL        ABBV  ...      YUM      ZBH     ZBRA      ZTS
Date                                                       ...                                    
2016-08-05   44.598419  33.196709   24.813429   47.136810  ...  2418532   780637   722700  4222000
2016-08-08   44.354244  33.109955   25.018902   46.995140  ...  4308344   758080   778400  5174600
2016-08-09   44.560852  33.341293   25.120481   47.292660  ...  2908164  1151334  1631900  3666700
2016-08-10   44.617195  32.946091   24.933479   47.058895  ...  3588085  4723580  1824800  5086400
2016-08-11   45.199451  33.697933   24.917318   47.427242  ...  3499478  2274961  1021600  3056900
...                ...        ...         ...         ...  ...      ...      ...      ...      ...
2024-07-29  137.380005  10.430000  218.240005  181.940002  ...  2150700  1202800   605800  1302900
2024-07-30  139.490005  10.760000  218.800003  186.779999  ...  1410500  1226600  1217400  2271300
2024-07-31  141.399994  10.640000  222.080002  185.320007  ...  1943100  1592800   634400  1740100
2024-08-01  142.860001  10.140000  218.360001  189.710007  ...  1627800  1129900   580900  1987800
2024-08-02  139.039993   9.630000  219.860001  189.289993  ...  2301500  1137700   447800  2365300

[2011 rows x 3018 columns]

In [27]:
df

Price        Adj Close                                     ...   Volume                           
Ticker               A        AAL        AAPL        ABBV  ...      YUM      ZBH     ZBRA      ZTS
date                                                       ...                                    
2016-08-05   44.598419  33.196709   24.813429   47.136810  ...  2418532   780637   722700  4222000
2016-08-08   44.354244  33.109955   25.018902   46.995140  ...  4308344   758080   778400  5174600
2016-08-09   44.560852  33.341293   25.120481   47.292660  ...  2908164  1151334  1631900  3666700
2016-08-10   44.617195  32.946091   24.933479   47.058895  ...  3588085  4723580  1824800  5086400
2016-08-11   45.199451  33.697933   24.917318   47.427242  ...  3499478  2274961  1021600  3056900
...                ...        ...         ...         ...  ...      ...      ...      ...      ...
2024-07-29  137.380005  10.430000  218.240005  181.940002  ...  2150700  1202800   605800  1302900
2024-07-30  139.490005  10.760000  218.800003  186.779999  ...  1410500  1226600  1217400  2271300
2024-07-31  141.399994  10.640000  222.080002  185.320007  ...  1943100  1592800   634400  1740100
2024-08-01  142.860001  10.140000  218.360001  189.710007  ...  1627800  1129900   580900  1987800
2024-08-02  139.039993   9.630000  219.860001  189.289993  ...  2301500  1137700   447800  2365300

[2011 rows x 3018 columns]

In [29]:
df.index.names = ["date", 'ticker']
df

Price               Adj Close       Close        High         Low        Open       Volume
date       ticker                                                                         
2016-08-05 A        44.598427   47.490002   47.540001   47.099998   47.099998    1250700.0
           AAL      33.196712   34.439999   34.720001   33.680000   33.709999   16437200.0
           AAPL     24.813435   26.870001   26.912500   26.545000   26.567499  162213600.0
           ABBV     47.136810   66.540001   66.599998   66.120003   66.150002    5045600.0
           ABT      39.097301   45.259998   45.410000   44.869999   45.000000    6555300.0
...                       ...         ...         ...         ...         ...          ...
2024-08-02 XYL     130.149994  130.149994  132.350006  128.850006  131.130005    1557100.0
           YUM     135.179993  135.179993  136.589996  133.000000  134.750000    2301500.0
           ZBH     110.360001  110.360001  111.930000  108.820000  111.220001    1137700.0
           ZBRA    322.299988  322.299988  332.130005  319.040009  331.799988     447800.0
           ZTS     180.589996  180.589996  183.529999  178.529999  181.600006    2365300.0

[990365 rows x 6 columns]

In [30]:
df.columns = df.columns.str.lower()
df

Price               adj close       close        high         low        open       volume
date       ticker                                                                         
2016-08-05 A        44.598427   47.490002   47.540001   47.099998   47.099998    1250700.0
           AAL      33.196712   34.439999   34.720001   33.680000   33.709999   16437200.0
           AAPL     24.813435   26.870001   26.912500   26.545000   26.567499  162213600.0
           ABBV     47.136810   66.540001   66.599998   66.120003   66.150002    5045600.0
           ABT      39.097301   45.259998   45.410000   44.869999   45.000000    6555300.0
...                       ...         ...         ...         ...         ...          ...
2024-08-02 XYL     130.149994  130.149994  132.350006  128.850006  131.130005    1557100.0
           YUM     135.179993  135.179993  136.589996  133.000000  134.750000    2301500.0
           ZBH     110.360001  110.360001  111.930000  108.820000  111.220001    1137700.0
           ZBRA    322.299988  322.299988  332.130005  319.040009  331.799988     447800.0
           ZTS     180.589996  180.589996  183.529999  178.529999  181.600006    2365300.0

[990365 rows x 6 columns]

In [31]:
df['garman_klass_vol'] = (np.log(df['high']) - np.log(df['high']))**2/2 - (2*np.log(2) - 1)*(np.log(df['adj close']) - np.log(df['open']))**2
df

Price               adj close       close        high  ...        open       volume  garman_klass_vol
date       ticker                                      ...                                           
2016-08-05 A        44.598427   47.490002   47.540001  ...   47.099998    1250700.0         -0.001151
           AAL      33.196712   34.439999   34.720001  ...   33.709999   16437200.0         -0.000091
           AAPL     24.813435   26.870001   26.912500  ...   26.567499  162213600.0         -0.001802
           ABBV     47.136810   66.540001   66.599998  ...   66.150002    5045600.0         -0.044359
           ABT      39.097301   45.259998   45.410000  ...   45.000000    6555300.0         -0.007637
...                       ...         ...         ...  ...         ...          ...               ...
2024-08-02 XYL     130.149994  130.149994  132.350006  ...  131.130005    1557100.0         -0.000022
           YUM     135.179993  135.179993  136.589996  ...  134.750000    2301500.0         -0.000004
           ZBH     110.360001  110.360001  111.930000  ...  111.220001    1137700.0         -0.000023
           ZBRA    322.299988  322.299988  332.130005  ...  331.799988     447800.0         -0.000326
           ZTS     180.589996  180.589996  183.529999  ...  181.600006    2365300.0         -0.000012

[990365 rows x 7 columns]

In [ ]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: )